<div style="display:block">
    <div style="width: 20%; display: inline-block; text-align: left;">
        <img src="http://upload.wikimedia.org/wikipedia/en/0/0c/Mu_Sigma_Logo.jpg" style="height:75px; margin-left:0px" />
    </div>
    <div style="width: 59%; display: inline-block">
        <h1  style="text-align: center">Consumer Promotion Optimization: An exercise in constrained optimization</h1>
        <div style="width: 100%; text-align: center; display: inline-block;"><i>Author:</i> <strong>Vivekanand Mishra</strong> </div>
    </div>
    <div style="width: 20%; text-align: right; display: inline-block;">
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Created: </i>
            <time datetime="2016-08-19" pubdate>May 22nd, 2018</time>
        </div>
        <div style="width: 100%; text-align: left; display: inline-block;">
            <i>Modified: </i>
            <time datetime="2016-08-19" pubdate>July 6th, 2018</time>
        </div>
    </div>
</div>

# Context

Mcinsey built a tool in R for a major beverage company to help them build promotion plans for the retailers through whom the company sells it's products.

At the core of the tool are two mixed models that predict the revenue/volume that is likely to be sold along with the number of households that company is likely to reach in the consumer market as a consequence of the offer plan that it rolls out. The documentation for this simulator had the following description of the mixed models:

<img src='cokeModels.png'>

This basically turns the tool into a scenario evaluation tool. The scenarios are created off-line - which are in the current use case basically promotion plans for a range of 8 kinds packages in the "sparkling" category for a duration of a quarter (13 weeks). The decision variables for each product pack and week are:

* The kind of offer to be rolled out - offer type
* The price point for the offer-product combination
* Whether the retailer is required to actively promote the offer tin their retail outlets

This amounts to a total of 546 parameters that need to be decided in order to build a promotion plan that satisfies the following constraints and requirements:

* The company must be able to derive a projected profit between 4.5-5.5% as a consequence of rolling out the plan.
* The retailer must have a non negative revenue lift.
* Active promotions cannot cumulatively cross more than 30 at a product-pack-week level

## Solution approaches taken 

__Approach__ : Irrespective of the algorithms used, at the core of all the solution attempts was to conceptualize the problem as a hyper-dimensional decision variable space where the interactions among these parameters affected the optimization objective in myriad and non-linear ways. Every solution approach in essence is the search for the optimal balance between cost and payoff within the space of these multitude of parameters.


### 1. Neural Networks 

This problem can be modeled as a multiple input and multiple output (MIMO) system where the input space is defined by the 546 parameters and the output space is defined by the KPIs.

The goal of this modeling exercise is to be able to formulate a way of querying the set of 546 parameters that can give rise to KPIs in certain range.



To solve this, problem first a neural network (multi layer perceptron) was built whose inputs were the 6 KPIs and the outputs are the 546 parameters that can be used to construct an offer plan (used inter-changeably with the word 'scenario' in this document)

Training data was generated by randomly sampling the constrained parameter space. Ten thousand scenarios were generated and their subsequent KPIs were calculated through a function that uses the scenarios to calculate more 
business related features and then making projections about the extent of household penetration and the revenue 
for the company. These results are further manipulated to arrive at the 6 KPIs that we care about.

This entire pipeline of taking a scenario (offer plan) and calculating the KPIs can be considered as a black box for our use case.

The neural network failed to perform well because of the limitations pertaining to data availability
and training infrastructure.  Another hypothesis being that the architecture was not good enough for modeling the dynamics of the system being modeled.



<img src="cpoNN.png" height="50">

In [1]:
library(reticulate)
PATH <- "/usr/bin/python3"
Sys.setenv(RETICULATE_PYTHON = PATH)
library(tensorflow)
library(kerasR)

successfully loaded keras


In [3]:
library(dplyr)
library(tidyr)
library(reshape2)
library(stringr)
library(lmerTest)
library(data.table)
library(progress)
library(doParallel)
library(foreach)
library(doSNOW)
library(xlsx)



In [4]:
allpromos <- c("Buy 2 Get 1", "Must Buy 2", "Must Buy 4", "Simple", "Buy 2 Get 2", 
    "Must Buy 3", "Must Buy 5")

In [5]:
packages <- c(".5L 6PK BOTTLE", "1.25L SINGLE BOTTLE", "12OZ 12PK CAN", "2L SINGLE BOTTLE", 
    "7.5OZ 10PK CAN", "7.5OZ 6PK CAN", "LARGE PK CAN", "12OZ 8PK PLS BOTTLE")
# dates in the 10k scenarios - NOTE single digit days do not have preceeding zero
# i.e. 8 not 08
dates <- c("10/4/2018", "10/11/2018", "10/18/2018", "10/25/2018", "11/1/2018", "11/8/2018", 
    "11/15/2018", "11/22/2018", "11/29/2018", "12/6/2018", "12/13/2018", "12/20/2018", 
    "12/27/2018")

In [6]:
constraints <- read.csv("https://mugitlab.mu-sigma.com/Interesting-Datasets/Structured-Data/tree/master/Retail/Simulation/constraint.csv", 
    stringsAsFactors = F)

In [7]:
mapRaw <- constraints[, c("package_size", "promotion", "comm_price")]
mapRawSpread <- mapRaw %>% group_by(package_size) %>% # map of all pack types with their corresponding valid offers and historic price
# ranges
summarise(promotion = list(unique(promotion)), comm_price = list(range(comm_price)))



In [9]:
set_path <- "./Version_3/"

In [10]:
if(Sys.info()['sysname'] == "Windows"){
    parent_dir <- stringr::str_replace_all(set_path,'\\Sim_App_ssd_sd\\\\Scenario_path_set','')
    parent_dir <- stringr::str_replace_all(parent_dir,'\\\\','/')
}else{
    parent_dir <- stringr::str_replace_all(set_path,'/Sim_App_ssd_sd/Scenario_path_set','')
    parent_dir <- stringr::str_replace_all(parent_dir,'\\\\','/')
}
source (paste0(parent_dir, "/Sim_App_ssd_sd/sim_app_ssd_v1.R"))

In [40]:
xvar <- readRDS("https://mugitlab.mu-sigma.com/Interesting-Datasets/Structured-Data/tree/master/Retail/Simulation/xvarList.RDS")
yvar <- readRDS("https://mugitlab.mu-sigma.com/Interesting-Datasets/Structured-Data/tree/master/Retail/Simulation/megalist.RDS")
y <- t(as.data.frame(yvar))
tempx <- t(as.data.frame(xvar))

xWhole <- tempx[, -c(2, 5)]
x <- xWhole

In [41]:
head(x)

,perc_chng_rev,perc_chng_coke_profit,perc_chng_hh,awr_mix_rate_change,cm_delta,dnnsi_rate_chnage.y
Sparkling.Soft.Drinks_Giant.Eagle_Scenario_10000.csv,-0.05000864,0.02613566,-0.03770666,0.07472507,-0.0093505809,0.08576268
Sparkling.Soft.Drinks_Giant.Eagle_Scenario_1000.csv,-0.05404725,0.01864123,-0.03056810,0.06996377,-0.0078011035,0.08446193
Sparkling.Soft.Drinks_Giant.Eagle_Scenario_1001.csv,-0.05799179,0.01117073,-0.05085492,0.07950535,0.0006010335,0.07044139
Sparkling.Soft.Drinks_Giant.Eagle_Scenario_1002.csv,-0.06711510,-0.01563052,-0.05037330,0.08630642,0.0033073982,0.07335568
Sparkling.Soft.Drinks_Giant.Eagle_Scenario_1003.csv,-0.03030506,0.03432243,-0.03106938,0.07045670,-0.0124229268,0.06420080
Sparkling.Soft.Drinks_Giant.Eagle_Scenario_1004.csv,-0.08537688,-0.03717105,-0.05299944,0.08194400,0.0134620214,0.07700039


In [42]:
mod <- Sequential()
mod$add(Dense(units = 16, input_shape = 6))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
mod$add(Dense(units = 32))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
mod$add(Dense(units = 64))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
mod$add(Dense(units = 128))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
mod$add(Dense(units = 256))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
mod$add(Dense(units = 512))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
mod$add(Dense(units = 546))
mod$add(Dropout(rate = 0.1))
mod$add(Activation("relu"))
keras_compile(mod, loss = "mse", optimizer = SGD(lr = 0.01, momentum = 0.2, decay = 0.001))
keras_fit(mod, x, y, batch_size = 10, epochs = 10, verbose = 1, validation_split = 0.01)

In [43]:
keras_save(mod,path = "kereasModel.h5")

In [44]:
packNumMap <- list()
packNumMap[[1]] <- 1
packNumMap[[2]] <- 79
packNumMap[[3]] <- 118
packNumMap[[4]] <- 196
packNumMap[[5]] <- 274
packNumMap[[6]] <- 339
packNumMap[[7]] <- 391
packNumMap[[8]] <- 469

In [45]:
ch_retail_rev <- data.frame(ch_retail_rev = seq(0, 1, 0.2))
ch_coke_rev <- data.frame(ch_coke_rev = seq(0, 1, 0.2))
ch_hh_cnt <- data.frame(ch_hh_cnt = seq(-1, 0, 0.2))
ch_coke_profit <- data.frame(ch_coke_profit = seq(4.5, 5.5, 0.2))
ch_mix_rate <- data.frame(ch_mix_rate = seq(0, 1, 0.2))
ch_cust_margin <- data.frame(ch_cust_margin = seq(0, 1, 0.2))
dnsi_12_oz <- data.frame(dnsi_12_oz = seq(4.5, 5.5, 0.2))

t1 <- merge(ch_retail_rev, ch_coke_rev, all = TRUE)
t2 <- merge(t1, ch_hh_cnt, all = TRUE)
t3 <- merge(t2, ch_coke_profit, all = TRUE)
t4 <- merge(t3, ch_mix_rate, all = TRUE)
t5 <- merge(t4, ch_cust_margin, all = TRUE)
t6 <- merge(t5, dnsi_12_oz, all = TRUE)

reqMatrix <- as.matrix(t5)
smallReqMatrix <- reqMatrix[sample(nrow(reqMatrix), 100, replace = FALSE), ]


model <- keras_load(path = "kereasModel.h5")

In [46]:
head(smallReqMatrix)

ch_retail_rev,ch_coke_rev,ch_hh_cnt,ch_coke_profit,ch_mix_rate,ch_cust_margin
0.8,0.8,0.0,5.3,0.8,1.0
0.4,0.4,-1.0,4.7,0.6,0.4
0.4,1.0,-1.0,5.5,0.4,1.0
0.0,0.2,-0.8,4.5,1.0,0.0
0.0,0.8,-0.2,5.5,0.2,0.2
0.8,0.6,0.0,5.5,0.6,1.0


In [47]:
rawPredictions <- keras_predict(model, smallReqMatrix, batch_size = 100, verbose = 1)

In [48]:
saveRDS(rawPredictions, 'rawPredictions.RDS')

In [51]:

packlist <- c(".5L 6PK BOTTLE", "1.25L SINGLE BOTTLE", "12OZ 12PK CAN", "2L SINGLE BOTTLE", 
    "7.5OZ 10PK CAN", "7.5OZ 6PK CAN", "LARGE PK CAN", "12OZ 8PK PLS BOTTLE")

template <- read.csv("https://mugitlab.mu-sigma.com/Interesting-Datasets/Structured-Data/tree/master/Retail/Simulation/template.csv", 
    stringsAsFactors = F)
rawPredLog <- list()
probPredLog <- list()
rawErrors <- list()
probErrosrs <- list()


for (predIndex in 1:dim(rawPredictions)[1]) {
    prediction <- rawPredictions[predIndex, ]
    for (packNum in 1:length(packlist)) {
        allowedPromos <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]
        
        cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]) + 2
        startindex <- packNumMap[[packNum]]
        for (weekNum in 1:length(dates)) {
            
            
            if (weekNum == 1) {
                currentPredVec <- prediction[startindex:((startindex - 1) + cellVeclength)]
            } else {
                currentPredVec <- prediction[((startindex - 1) + ((weekNum - 1) * 
                  cellVeclength) + 1):((startindex - 1) + ((weekNum - 1) * cellVeclength) + 
                  cellVeclength)]
                
            }
            
            packSubVec <- currentPredVec[1:(cellVeclength - 2)]
            
            if (length(which(packSubVec == max(packSubVec))) > 1) {
                print(packSubVec)
                template[which(template$package_size == packlist[packNum] & template$week == 
                  dates[weekNum]), "promotion"] <- allowedPromos[sample.int(length(allowedPromos), 
                  1)]
            } else {
                template[which(template$package_size == packlist[packNum] & template$week == 
                  dates[weekNum]), "promotion"] <- allowedPromos[which(packSubVec == 
                  max(packSubVec))]
            }
            template[which(template$package_size == packlist[packNum] & template$week == 
                dates[weekNum]), "comm_price"] <- currentPredVec[cellVeclength - 
                1]  #*10
            template[which(template$package_size == packlist[packNum] & template$week == 
                dates[weekNum]), "feature"] <- currentPredVec[cellVeclength]
            
        }
    }
    write.csv(template, paste0(parent_dir, "Sim_App_ssd_sd/Input/template.csv"))
    returendScenario <- sim_app_ssd("template.csv")
    rawPredLog[[predIndex]] <- list(scenario = template, reqSpecs = smallReqMatrix[[predIndex]], 
        returnedSpecs = returendScenario)
    
}

[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 0 0 0 0
[1] "=============================================="
[1] "template.csv"
[1] "+++++++++++++++++++++++++++++++++"
[1] 0 0
[1] 0 0
[1] 

In [52]:
saveRDS(rawPredLog, 'rawPredLog.RDS')

In [56]:
rawPredLog[[1]]$reqSpecs
rawPredLog[[1]]$returnedSpecs

[1] 0.8

scenario,x,% Change in Retail Revenue,% Change in Coke Revenue,% Change in HH Count,% Change in Coke Profit,%change in mix rate,% change in Cus Margin,12oz_12pk_dnnsi_rate_chnage
template.csv,104,-1,6.695982,5.13754,7.74134,0.05878093,-Inf,0.0402808


### 2 - Genetic algorithm

Prolific optimization techniques like linear programming cannot be used in this case becuase despite the constraints being linear because the problem should not be sated as an LPP owing to the line depicting the problem changing with every scenario.

Having been only partially successful while using gradient based optimization on this MIMO problem. A gradient free optimization techniqe was employed.

The algoritm of choice for this is [Differential Evolution](https://en.wikipedia.org/wiki/Differential_evolution).

This was done using the Deoptim package in R - The package has bugs - and so the Genetic Algorithm route was taken


The outline of this approach revolves around the following main requirement:

Posing the problem as a constrained optimization exercise and converting the decision variables into the parameters and constraints in such a way that the search space for the parameters is programmable into the the algorithm API.



For posing the problem as an optimization exercise, a single KPI to be optmized had to be zeroed in on. This turns out to be a function of the predictions made by the mixed models in the black-box and various calculations obtained by performing subsequent operations on them. It was also noticed that all the formulas for the KPIs being calculated were directly proportional to the the mixed model predictions. Thus, it was concluded that of the inputs to these mixed models could be found in such a way that the combined output of these models was maximized and so we had our objective function required to perform constrained optimization.

The constraints were a function of the business rules around the formulation of a promotion plan.

The following business constraints were to be incorporated:

1. One requirement of the plan was to come up with a publicity schedule, and the total number number of week-product publicity recommendations could not exceed 30. This was out of the 104 possible week-product plans that had to be designed.

2. Every product has a range of allowed prices that the products could be offered at . 104 of such decision variables exist.

3. Every product can only accommodate a certain subspace of all possible promotion plans. 

This amounts to a total of 546 variables and their constraints.

The code inside the black box function was modified so the only output it returns is the sum of the combined predictions of the two mixed models. This was added to the overall objective function where not satisfying constraints is penalized.

In [2]:
library(GA)
library(dplyr)
library(tidyr)
library(reshape2)
library(stringr)
library(lmerTest)
library(data.table)
library(progress)
library(doParallel)
library(foreach)
library(doSNOW)
library(xlsx)




parent_dir <<- "./Version_3/"
source(paste0(parent_dir, "Sim_App_ssd_sd/sim_app_ssd_optimization.R"))
constraints <<- read.csv("https://mugitlab.mu-sigma.com/Interesting-Datasets/Structured-Data/tree/master/Retail/Simulation/constraint.csv", 
    stringsAsFactors = F)
packlist <<- c(".5L 6PK BOTTLE", "1.25L SINGLE BOTTLE", "12OZ 12PK CAN", "2L SINGLE BOTTLE", 
    "7.5OZ 10PK CAN", "7.5OZ 6PK CAN", "LARGE PK CAN", "12OZ 8PK PLS BOTTLE")
allpromos <<- unique(constraints$promotion)

mapRaw <<- constraints[, c("package_size", "promotion", "comm_price")]

mapRawSpread <<- mapRaw %>% group_by(package_size) %>% summarise(promotion = list(unique(promotion)), 
    comm_price = list(range(comm_price)))


dates <<- c("10/4/2018", "10/11/2018", "10/18/2018", "10/25/2018", "11/1/2018", "11/8/2018", 
    "11/15/2018", "11/22/2018", "11/29/2018", "12/6/2018", "12/13/2018", "12/20/2018", 
    "12/27/2018")
template <<- read.csv("https://mugitlab.mu-sigma.com/Interesting-Datasets/Structured-Data/tree/master/Retail/Simulation/template.csv", 
    stringsAsFactors = F)

packNumMap <<- list()
packNumMap[[1]] <- 1
packNumMap[[2]] <- 79
packNumMap[[3]] <- 118
packNumMap[[4]] <- 196
packNumMap[[5]] <- 274
packNumMap[[6]] <- 339
packNumMap[[7]] <- 391
packNumMap[[8]] <- 469



lowerBounds <- c()
for (packNum in 1:length(packlist)) {
    for (weekNum in 1:length(dates)) {
        cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]) + 2
        startindex <- packNumMap[[packNum]]
        lowPrice <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "comm_price"]$comm_price[[1]][1]
        lowerBounds <- c(lowerBounds, rep(0, (cellVeclength - 2)))  #packs
        lowerBounds <- c(lowerBounds, lowPrice)  # price
        lowerBounds <- c(lowerBounds, 0)  # feature 
        
    }
}

upperBounds <- c()

for (packNum in 1:length(packlist)) {
    for (weekNum in 1:length(dates)) {
        cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]) + 2
        startindex <- packNumMap[[packNum]]
        highPrice <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "comm_price"]$comm_price[[1]][2]
        upperBounds <- c(upperBounds, rep(1, (cellVeclength - 2)))  # packs
        upperBounds <- c(upperBounds, highPrice)  # price
        upperBounds <- c(upperBounds, 1)  # feature 
        
    }
}



f = function(x) {
    prediction <- x
    for (packNum in 1:length(packlist)) {
        allowedPromos <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]
        
        cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]) + 2
        startindex <- packNumMap[[packNum]]
        for (weekNum in 1:length(dates)) {
            
            if (weekNum == 1) {
                currentPredVec <- prediction[startindex:((startindex - 1) + cellVeclength)]
            } else {
                currentPredVec <- prediction[((startindex - 1) + ((weekNum - 1) * 
                  cellVeclength) + 1):((startindex - 1) + ((weekNum - 1) * cellVeclength) + 
                  cellVeclength)]
            }
            packSubVec <- currentPredVec[1:(cellVeclength - 2)]
            if (length(which(packSubVec == max(packSubVec))) > 1) {
                # print(packSubVec)
                template[which(template$package_size == packlist[packNum] & template$week == 
                  dates[weekNum]), "promotion"] <- allowedPromos[sample.int(length(allowedPromos), 
                  1)]
            } else {
                template[which(template$package_size == packlist[packNum] & template$week == 
                  dates[weekNum]), "promotion"] <- allowedPromos[which(packSubVec == 
                  max(packSubVec))]
            }
            template[which(template$package_size == packlist[packNum] & template$week == 
                dates[weekNum]), "comm_price"] <- currentPredVec[cellVeclength - 
                1]  #*10
            template[which(template$package_size == packlist[packNum] & template$week == 
                dates[weekNum]), "feature"] <- currentPredVec[cellVeclength]
            
        }
    }
    write.csv(template, paste0(parent_dir, "Sim_App_ssd_sd/Input/template.csv"), 
        row.names = FALSE)
    returendScenario <- sim_app_ssd("template.csv")
    
    return(-1 * returendScenario)
    
}



eqConstrIndices <<- c()
for (packNum in 1:length(packlist)) {
    for (weekNum in 1:length(dates)) {
        cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]) + 2
        startindex <- packNumMap[[packNum]]
        if (weekNum == 1) {
            eqConstrIndices <- c(eqConstrIndices, c(startindex:((startindex - 1) + 
                (cellVeclength - 2))))
            
        } else {
            eqConstrIndices <- c(eqConstrIndices, c(startindex:((startindex - 1) + 
                (cellVeclength - 2))))
            
        }
        
    }
}



featureIndex <<- c()
for (packNum in 1:length(packlist)) {
    for (weekNum in 1:length(dates)) {
        cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
            "promotion"]$promotion[[1]]) + 2
        startindex <- packNumMap[[packNum]]
        if (weekNum == 1) {
            featureIndex <- c(featureIndex, ((startindex - 1) + cellVeclength))
        } else {
            featureIndex <- c(featureIndex, ((startindex - 1) + ((weekNum - 1) * 
                cellVeclength) + cellVeclength))
        }
    }
}







fitness <- function(x) {
    f <- f(x)
    
    
    eqConstr <- abs(104 - sum(x[eqConstrIndices]))
    featConstr <- abs(30 - (sum(x[featureIndex])))
    lowPriceVec <- c()  #if everything in this is negative  -> we good
    for (packNum in 1:length(packlist)) {
        for (weekNum in 1:length(dates)) {
            cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == 
                packlist[packNum]), "promotion"]$promotion[[1]]) + 2
            startindex <- packNumMap[[packNum]]
            lowPrice <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
                "comm_price"]$comm_price[[1]][1]
            if (weekNum == 1) {
                lowPriceVec <- c(lowPriceVec, lowPrice - x[((startindex - 1) + cellVeclength) - 
                  1])
            } else {
                lowPriceVec <- c(lowPriceVec, lowPrice - x[((startindex - 1) + ((weekNum - 
                  1) * cellVeclength) + cellVeclength) - 1])
            }
            
        }
    }
    highPriceVec <- c()  #if everything in this is negative  -> we good
    for (packNum in 1:length(packlist)) {
        for (weekNum in 1:length(dates)) {
            cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == 
                packlist[packNum]), "promotion"]$promotion[[1]]) + 2
            startindex <- packNumMap[[packNum]]
            highPrice <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
                "comm_price"]$comm_price[[1]][2] + 1
            if (weekNum == 1) {
                highPriceVec <- c(highPriceVec, x[((startindex - 1) + cellVeclength) - 
                  1] - highPrice)
            } else {
                highPriceVec <- c(highPriceVec, x[((startindex - 1) + ((weekNum - 
                  1) * cellVeclength) + cellVeclength) - 1] - highPrice)
            }
            
            
        }
    }
    
    sumLpVec <- sum(lowPriceVec)
    sumHpVec <- sum(highPriceVec)
    penalty <- eqConstr + featConstr - sumLpVec - sumHpVec
    
    return(f - penalty)
}



GA <- ga("real-valued", fitness = fitness, lower = lowerBounds, upper = upperBounds, 
    maxiter = 10, run = 200, seed = 123)



[1] "template.csv"
[1] 6.453551
[1] "template.csv"
[1] 9.264743
[1] "template.csv"
[1] 6.518238
[1] "template.csv"
[1] 10.21456
[1] "template.csv"
[1] 8.484224
[1] "template.csv"
[1] 8.877255
[1] "template.csv"
[1] 9.018233
[1] "template.csv"
[1] 12.3792
[1] "template.csv"
[1] 7.261009
[1] "template.csv"
[1] 14.45163
[1] "template.csv"
[1] 8.007888
[1] "template.csv"
[1] 6.80052
[1] "template.csv"
[1] 5.179956
[1] "template.csv"
[1] 8.385174
[1] "template.csv"
[1] 10.02077
[1] "template.csv"
[1] 12.29694
[1] "template.csv"
[1] 8.694405
[1] "template.csv"
[1] 6.026606
[1] "template.csv"
[1] 8.155969
[1] "template.csv"
[1] 13.41824
[1] "template.csv"
[1] 8.287108
[1] "template.csv"
[1] 9.237221
[1] "template.csv"
[1] 7.150262
[1] "template.csv"
[1] 8.608397
[1] "template.csv"
[1] 9.255654
[1] "template.csv"
[1] 8.641263
[1] "template.csv"
[1] 5.480479
[1] "template.csv"
[1] 9.931216
[1] "template.csv"
[1] 11.78212
[1] "template.csv"
[1] 7.817724
[1] "template.csv"
[1] 14.54675
[1] "templ

In [5]:
GA@solution

x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,⋯,x537,x538,x539,x540,x541,x542,x543,x544,x545,x546
0.2357091,0.2516947,0.3840101,0.3423076,3.170362,0.5456543,0.3059275,0.5750003,0.3012255,0.6421712,⋯,0.6415231,0.5487809,3.544169,0.7742847,0.7984015,0.3925362,0.5372969,0.4417976,3.713271,0.4074425


The following code converts the vector of 546 parameters into a scenario after optimization

In [6]:
prediction <- GA@solution
for (packNum in 1:length(packlist)) {
    allowedPromos <- mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
        "promotion"]$promotion[[1]]
    
    cellVeclength <- length(mapRawSpread[which(mapRawSpread$package_size == packlist[packNum]), 
        "promotion"]$promotion[[1]]) + 2
    startindex <- packNumMap[[packNum]]
    for (weekNum in 1:length(dates)) {
        
        
        if (weekNum == 1) {
            currentPredVec <- prediction[startindex:((startindex - 1) + cellVeclength)]
        } else {
            currentPredVec <- prediction[((startindex - 1) + ((weekNum - 1) * cellVeclength) + 
                1):((startindex - 1) + ((weekNum - 1) * cellVeclength) + cellVeclength)]
        }
        packSubVec <- currentPredVec[1:(cellVeclength - 2)]
        if (length(which(packSubVec == max(packSubVec))) > 1) {
            print(packSubVec)
            template[which(template$package_size == packlist[packNum] & template$week == 
                dates[weekNum]), "promotion"] <- allowedPromos[sample.int(length(allowedPromos), 
                1)]
        } else {
            template[which(template$package_size == packlist[packNum] & template$week == 
                dates[weekNum]), "promotion"] <- allowedPromos[which(packSubVec == 
                max(packSubVec))]
        }
        template[which(template$package_size == packlist[packNum] & template$week == 
            dates[weekNum]), "comm_price"] <- currentPredVec[cellVeclength - 1]  #*10
        template[which(template$package_size == packlist[packNum] & template$week == 
            dates[weekNum]), "feature"] <- currentPredVec[cellVeclength]
    }
}

# Optimal scenario

__Note__: Solution will improve with more iterations

In [7]:
template

category,package_size,cta,promotion,week,edv,comm_price,cases_on_dis,feature,dnnsi,list_price,on_invoice_trade,off_invoice_trade,cogs,gross_profit,Holiday_Week_name,Holiday_week_type
Sparkling Soft Drinks,7.5OZ 10PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,11/15/2018,4.19,3.6572113,0,0.55228890,8.056627,11.04,2.59,0.89800,4.619588,2.932412,Thanksgiving,Holiday
Sparkling Soft Drinks,7.5OZ 6PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,11/29/2018,3.29,0.5171850,0,0.33311571,7.551226,11.04,1.00,2.95997,4.020664,3.059366,0,0
Sparkling Soft Drinks,LARGE PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,12/13/2018,8.49,0.6378096,0,0.56713890,4.821333,12.24,4.34,0.66800,4.898598,2.333402,Pre-Christmas,Pre-holiday
Sparkling Soft Drinks,7.5OZ 6PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,12/13/2018,3.29,0.2617297,0,0.49943610,7.551226,11.04,1.00,2.95997,4.020664,3.059366,Pre-Christmas,Pre-holiday
Sparkling Soft Drinks,7.5OZ 10PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,12/6/2018,4.19,4.2081680,0,0.64274718,8.056627,11.04,2.59,0.89800,4.619588,2.932412,0,0
Sparkling Soft Drinks,7.5OZ 6PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,10/18/2018,3.29,0.2371217,0,0.63045010,7.551226,11.04,1.00,2.95997,4.020664,3.059366,Pre-Halloween,Pre-holiday
Sparkling Soft Drinks,7.5OZ 10PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,12/20/2018,4.19,3.5750387,0,0.45706339,8.056627,11.04,2.59,0.89800,4.619588,2.932412,Christmas,Holiday
Sparkling Soft Drinks,7.5OZ 10PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 3,10/25/2018,4.19,3.2885891,0,0.42972073,8.056627,11.04,2.59,0.89800,4.619588,2.932412,Halloween,Holiday
Sparkling Soft Drinks,LARGE PK CAN,Giant Eagle: GIANT EAGLE TOTAL CTA,Must Buy 2,11/8/2018,8.49,0.5218174,0,0.38671756,4.821333,12.24,4.34,0.66800,4.898598,2.333402,Pre-Thanksgiving,Pre-holiday
Sparkling Soft Drinks,2L SINGLE BOTTLE,Giant Eagle: GIANT EAGLE TOTAL CTA,Simple,11/29/2018,2.29,1.6089246,0,0.15817037,4.287337,15.84,2.16,1.60000,5.970000,6.110000,0,0


# Conclusion

* The neural network based approach may require more data and experimentation in order to start generating usable results - but once constructed, it will have the advantage of being able to analyze any scenario and thus make optimal directional decisions guided more closely by business logic.

* Genetic algorithm can be used to find the optimal solution. Multiple iterations may be tried to find if there are multiple minima that are viable as solutions. The main disadvantage here is that unlike neural networks, the framework does not make it easy to run what-if scenarios conveniently.